In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_tablas = pickle.load(open('dict_tablas','rb'))
df_historical_data = pd.read_csv(r'Datasets\Clean_Fifa_WorldCup_Matches.csv')
df_fixture = pd.read_csv(r'Datasets\Fifa_WorldCup_Fixture.csv')

In [3]:
dict_tablas['Grupo F']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Belgium,0,0,0,0,0,0,0,0
1,2,Canada,0,0,0,0,0,0,0,0
2,3,Morocco,0,0,0,0,0,0,0,0
3,4,Croatia,0,0,0,0,0,0,0,0


## 1.- Calcular Nivel de equipo

In [4]:
#Dividir df en df_home y df_away
df_historical_data


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
895,Russia,Croatia,2018,2,2
896,France,Belgium,2018,1,0
897,Croatia,England,2018,2,1
898,Belgium,England,2018,2,0


In [5]:
#Dividir df en df_home y df_away
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [6]:
#Renombrar Columnas
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalScored','AwayGoals':'GoalConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalConceded','AwayGoals':'GoalScored'})

In [7]:
#Concatenar df_home y df_away, hacer grupo por team y calcular promedio
df_team_strenght = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

#Convertir la primera letra a minúscula
df_team_strenght.info()
df_team_strenght.head()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Algeria to Zaire
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GoalScored    85 non-null     float64
 1   GoalConceded  85 non-null     float64
dtypes: float64(2)
memory usage: 2.0+ KB


,GoalScored,GoalConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690


In [8]:
def prediction_points(home,away):
    if home in df_team_strenght.index and away in df_team_strenght.index:
        #Para obtener lambda vamos a multiplicar el promedio de goles anotas del
        #equipo rival por el promedio de goles recibidos del equipo visitante
        #goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home, 'GoalScored'] * df_team_strenght.at[away,'GoalConceded']
        lamb_away = df_team_strenght.at[away, 'GoalScored'] * df_team_strenght.at[home,'GoalConceded']
        prob_home, prob_away, prob_draw = 0,0,0
        for x in range(0,11): #Number of goals home team
            for y in range(0,11): #Number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0,0)

## 2.1- Probamos nuestra función

In [9]:
#Testear con partidos: Argentina - México, England - United States, Qatar(H) -Ecuador
prediction_points('argentina','mexico')
prediction_points('england','united states')
#prediction_points('Qatar','Ecuador')

(0, 0)

## 3.- Prediciendo Resultados de Mundial

In [10]:
#Dividiendo fixture en grupo, octavos, cuartos,...
df_fixture_gruop_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semis = df_fixture[60:62].copy()
df_fixture_third_place = df_fixture[62:63].copy()
df_fixture_final = df_fixture[63:].copy()

## 3.1- Prediciendo Grupos

In [11]:
#Correr todos los partidos de la fase de grupos y actualizamos las tablas de cada grupo
for group in dict_tablas:
    print(f"Procesando grupo: {group}")
    teams_in_group = dict_tablas[group]['Team'].values
    print(f"Equipos en el grupo: {teams_in_group}")
    
    # Verificar contenido de teams_in_group
    print("Contenido de teams_in_group:")
    for team in teams_in_group:
        print(f"'{team}'")
    
    teams_in_group = [team.strip().lower() for team in dict_tablas[group]['Team'].values]
    df_fixture_gruop_48['Home'] = df_fixture_gruop_48['Home'].str.strip().str.lower()
    df_fixture_gruop_48['Away'] = df_fixture_gruop_48['Away'].str.strip().str.lower()

    df_fixture_gruop_6 = df_fixture_gruop_48[df_fixture_gruop_48['Home'].isin(teams_in_group)]
    #print(f"Fixture filtrado para el grupo: {df_fixture_gruop_6}")
    
    if df_fixture_gruop_6.empty:
        print(f"No hay partidos para los equipos en el grupo: {group}")
        continue
    
    for index, row in df_fixture_gruop_6.iterrows():
        home, away = row['Home'].capitalize(), row['Away'].capitalize()
        #print(f"Partido: {home} vs {away}")
        
        points_home, points_away = prediction_points(home, away)
        #print(f"Puntos predichos - Home: {points_home}, Away: {points_away}")
        
        dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
        dict_tablas[group].loc[dict_tablas[group]['Team'] == away, 'Pts'] += points_away
    
    dict_tablas[group] = dict_tablas[group].sort_values('Pts', ascending=False).reset_index(drop=True)
    dict_tablas[group] = dict_tablas[group][['Team','Pts']]
    dict_tablas[group] = dict_tablas[group].round(0)
    #print(f"Tabla actualizada para el grupo {group}:")
    print(dict_tablas[group])

Procesando grupo: Grupo A
Equipos en el grupo: ['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
Contenido de teams_in_group:
'Qatar (H)'
'Ecuador'
'Senegal'
'Netherlands'
          Team  Pts
0  Netherlands  4.0
1      Senegal  2.0
2      Ecuador  2.0
3    Qatar (H)  0.0
Procesando grupo: Grupo B
Equipos en el grupo: ['England' 'Iran' 'United States' 'Wales']
Contenido de teams_in_group:
'England'
'Iran'
'United States'
'Wales'
            Team  Pts
0        England  4.0
1          Wales  3.0
2           Iran  1.0
3  United States  0.0
Procesando grupo: Grupo C
Equipos en el grupo: ['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
Contenido de teams_in_group:
'Argentina'
'Saudi Arabia'
'Mexico'
'Poland'
           Team  Pts
0     Argentina  4.0
1        Poland  3.0
2        Mexico  1.0
3  Saudi Arabia  0.0

C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.8966936]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.38825101]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.86042645]' has dty


Procesando grupo: Grupo D
Equipos en el grupo: ['France' 'Australia' 'Denmark' 'Tunisia']
Contenido de teams_in_group:
'France'
'Australia'
'Denmark'
'Tunisia'
        Team  Pts
0     France  7.0
1    Denmark  6.0
2    Tunisia  3.0
3  Australia  2.0
Procesando grupo: Grupo E
Equipos en el grupo: ['Spain' 'Costa Rica' 'Germany' 'Japan']
Contenido de teams_in_group:
'Spain'
'Costa Rica'
'Germany'
'Japan'
         Team  Pts
0     Germany  4.0
1       Spain  3.0
2       Japan  1.0
3  Costa Rica  0.0
Procesando grupo: Grupo F
Equipos en el grupo: ['Belgium' 'Canada' 'Morocco' 'Croatia']
Contenido de teams_in_group:
'Belgium'
'Canada'
'Morocco'
'Croatia'


C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.1926621]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39879893]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.71189919]' has dty

      Team  Pts
0  Croatia  7.0
1  Belgium  6.0
2  Morocco  4.0
3   Canada  0.0
Procesando grupo: Grupo G
Equipos en el grupo: ['Brazil' 'Serbia' 'Switzerland' 'Cameroon']
Contenido de teams_in_group:
'Brazil'
'Serbia'
'Switzerland'
'Cameroon'
          Team  Pts
0       Brazil  8.0
1  Switzerland  4.0
2       Serbia  3.0
3     Cameroon  2.0
Procesando grupo: Grupo H
Equipos en el grupo: ['Portugal' 'Ghana' 'Uruguay' 'South Korea']
Contenido de teams_in_group:
'Portugal'
'Ghana'
'Uruguay'
'South Korea'
          Team  Pts
0     Portugal  4.0
1      Uruguay  3.0
2        Ghana  2.0
3  South Korea  0.0


C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.06786741]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_8612\3960421313.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.94865532]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home


In [12]:
#mostrar tabla actualizada
dict_tablas['Grupo A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


## 3.2 Prediciendo Octavos de final